<a href="https://colab.research.google.com/github/FarshadAmiri/Structural_Engineering/blob/master/HPO_1_Mahdi_Structures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import GridSearchCV
import time

In [2]:
Dataset = pd.read_excel(r'https://github.com/FarshadAmiri/Structural_Engineering/blob/master/D600.xlsx?raw=true')
cols = ['Mu33','Mu22','Pu','L','Cb','Delta']

for col in cols:
    col_zscore = col + '_zscore'
    Dataset[col_zscore] = (Dataset[col] - Dataset[col].mean())/Dataset[col].std(ddof=0)

In [3]:
#Defining Trainig data and Test Data
shuffled_dataset = Dataset.reindex(np.random.permutation(Dataset.index))

train_df = shuffled_dataset[:500]
test_df  = shuffled_dataset[500:]

In [4]:
#Trying 'Z score'
x = np.column_stack((train_df.Mu33_zscore.values , train_df.Mu22_zscore.values, train_df.Pu_zscore.values, train_df.L_zscore.values, train_df.Cb_zscore.values, train_df.Delta_zscore.values, train_df.L1tb.values))
y = np.column_stack((train_df.h.values, train_df.b.values, train_df.tf.values, train_df.tw.values))

test_x = np.column_stack((test_df.Mu33_zscore.values , test_df.Mu22_zscore.values, test_df.Pu_zscore.values, test_df.L_zscore.values, test_df.Cb_zscore.values, test_df.Delta_zscore.values,  test_df.L1tb.values))
test_y = np.column_stack((test_df.h.values, test_df.b.values, test_df.tf.values, test_df.tw.values))

In [5]:
# Function to create model; used by KerasRegressor
def create_model(Lambda,neurons,LR):
	# create model
  model = keras.Sequential([
      keras.layers.Dense(neurons/16,input_shape=(7,),activation='relu',kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
      keras.layers.Dense(neurons/8,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
      keras.layers.Dense(neurons/4,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
      keras.layers.Dense(neurons/2,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
      keras.layers.Dense(neurons,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
      keras.layers.Dense(neurons/2,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
      keras.layers.Dense(neurons/4,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
      keras.layers.Dense(neurons/8,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
      keras.layers.Dense(neurons/16,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda))])
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError(name='rmse', dtype=None))
  return model

In [6]:
# Create model
model = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=create_model 
                                              # ,**sk_params
                                              )

# Define the grid serach parameters
# neurons = [128,256,512]
# Lambda = [0.001,0.005,0.01,0.015]
# LR = [0.001,0.0015,0.002]
# batch_size = [6,8,10]

neurons = [128,256]
Lambda = [0.001,0.01]
LR = [0.001,0.002]
batch_size = [6]

param_grid = dict(neurons = neurons ,Lambda = Lambda, LR=LR
                  ,batch_size=batch_size
                  )
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(x,y
                      #  ,batch_size=6
                       , epochs =50 
                      #  ,validation_split=0.4
                       )


Epoch 1/50


ValueError: ignored

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))